link to Kaggle:
https://www.kaggle.com/datasets/wcukierski/enron-email-dataset/data

git link:
https://github.com/MaorEl/cyber-ai

In [10]:
import pandas as pd
import numpy as np

In [4]:
enron_df = pd.read_pickle('enron_students.pkl')
enron_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Date            517401 non-null  object
 1   From            517401 non-null  object
 2   To              495554 non-null  object
 3   X-To            517372 non-null  object
 4   X-From          517372 non-null  object
 5   X-cc            517372 non-null  object
 6   X-bcc           517372 non-null  object
 7   Subject         517401 non-null  object
 8   email_body      517401 non-null  object
 9   verdict         517401 non-null  object
 10  violated_rules  517401 non-null  object
dtypes: object(11)
memory usage: 43.4+ MB


In [5]:
enron_df.head(10)

,Date,From,To,X-To,X-From,X-cc,X-bcc,Subject,email_body,verdict,violated_rules
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,Tim Belden <Tim Belden/Enron@EnronXGate>,Phillip K Allen,,,,Here is our forecast\n\n,ALLOW,
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,John J Lavorato <John J Lavorato/ENRON@enronXg...,Phillip K Allen,,,Re:,Traveling to have a business meeting takes the...,ALLOW,
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Leah Van Arsdall,Phillip K Allen,,,Re: test,test successful. way to go!!!,ALLOW,
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,Randall L Gay,Phillip K Allen,,,,"Randy,\n\n Can you send me a schedule of the s...",ALLOW,
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Greg Piper,Phillip K Allen,,,Re: Hello,Let's shoot for Tuesday at 11:45.,ALLOW,
5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Greg Piper,Phillip K Allen,,,Re: Hello,"Greg,\n\n How about either next Tuesday or Thu...",ALLOW,
6,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",phillip.allen@enron.com,"david.l.johnson@enron.com, john.shafer@enron.com","david.l.johnson@enron.com, John Shafer",Phillip K Allen,,,,Please cc the following distribution list with...,ALLOW,
7,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",phillip.allen@enron.com,joyce.teixeira@enron.com,Joyce Teixeira,Phillip K Allen,,,Re: PRC review - phone calls,any morning between 10 and 11:30,ALLOW,
8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Mark Scott,Phillip K Allen,,,Re: High Speed Internet Access,1. login: pallen pw: ke9davis\n\n I don't thi...,BLOCK,2.3
9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",phillip.allen@enron.com,zimam@enron.com,zimam@enron.com,Phillip K Allen,,,FW: fixed forward or other Collar floor gas pr...,---------------------- Forwarded by Phillip K ...,BLOCK,2.1


We want to check the number of emails that are send, to understand our approach in enforcing the policies

In [6]:
import re

def extract_emails(text):
    if pd.isna(text):
        return []
    # Regex to match email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.findall(email_pattern, text)

def max_emails_per_column(column):
    # Extract emails from each cell in the column, count them, and find the index with the maximum number
    email_counts = column.apply(lambda x: len(extract_emails(x)))
    max_emails = email_counts.max()
    max_row = email_counts.idxmax()
    return max_emails, max_row

# Calculate the maximum number of emails for each column
max_emails_column1 = max_emails_per_column(enron_df['To'])
max_emails_column2 = max_emails_per_column(enron_df['X-To'])
max_emails_column3 = max_emails_per_column(enron_df['X-cc'])
max_emails_column4 = max_emails_per_column(enron_df['X-bcc'])


print(f"Maximum number of emails in 'To': {max_emails_column1[0]}")
print(f"Maximum number of emails in 'X-To': {max_emails_column2[0]}")
print(f"Maximum number of emails in 'X-cc': {max_emails_column3[0]}")

Maximum number of emails in 'To': 1029
Maximum number of emails in 'X-To': 622
Maximum number of emails in 'X-cc': 565


In [7]:

# TODO: need to check if we need to drop rows with null values in 'To' column.
def count_nulls(df, columns):
    """Print the number of nulls in specified columns of a dataframe."""
    for column in columns:
        null_count = df[column].isnull().sum()
        print(f"Number of rows with null values in '{column}': {null_count}")

def drop_nulls(df, columns):
    """Drop rows from the dataframe where specified columns have null values."""
    before_drop = df.shape[0]
    df.dropna(subset=columns, inplace=True)
    after_drop = df.shape[0]
    print(f"Dropped {before_drop - after_drop} rows. Remaining rows: {after_drop}")

def apply_lower_case(df, columns):
    """Convert the contents of specified columns in the dataframe to lower case."""
    for column in columns:
        df[column] = df[column].str.lower()

# replace empty missing value in subject with np.nan
def replace_empty_with_nan(subject):
    column=[]
    for sub in subject:
        if sub=="":
            column.append(np.nan)
        else:
            column.append(sub)
    return column


def process_dataframe(df):
    # Identify columns to check for nulls and to convert to lower case inplace!
    columns_to_check = ['From', 'To']
    email_related_columns = ['From', 'To', 'X-From', 'X-To', 'X-cc', 'X-bcc']
    
    # Replace empty strings with NaN
    for col in email_related_columns:
        df[col] = df[col].replace('', np.nan)

    # Null removal process
    count_nulls(df, columns_to_check)
    drop_nulls(df, columns_to_check)
    
    # Convert email related columns to lower case (email is case insensitive)
    apply_lower_case(df, email_related_columns)
    

In [8]:
from datetime import datetime
from dateutil import parser

def parse_date(date_str):
    # Parse the datetime string including timezone using dateutil.parser
    dt = parser.parse(date_str)
    
    # Extract components
    day_of_week = dt.strftime('%A')
    day = dt.day
    month = dt.month
    year = dt.year
    time_in_minutes = (dt.hour * 60 + dt.minute) / 1440  # Convert time to minutes and normalize
    timezone = dt.tzname()  # Gets the timezone name
    
    return day_of_week, day, month, year, time_in_minutes, timezone


def get_email_type(text):
    """
    Function to determine the type of email based on the subject line.
    Forwarded emails start with 'Fo', replies start with 'Re', and other emails are 'Normal'.
    """
    if text.startswith('Fo') or text.startswith('Re') or text.startswith('FW'):
        return text[:2]
    else:
        return 'Normal'

def split_email_addresses(line):
    """
    Function to split email addresses in the 'From' and 'To' columns.
    The email addresses are separated by commas.
    TODO: need to add regex to extract email addresses from the text.
    """

    if line:
        addrs = line.split(',')
        for i, addr in enumerate(addrs):
            # Remove any names in the email addresses
            if '<' in addr:
                pass
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs



def transform_dataframe(df, split_date=True):
    # transform the 'Date' column to extract day of week, day, month, year, time in minutes, and timezone
    if split_date:
        df['day_of_week'], df['day'], df['month'], df['year'], df['time_in_minutes'], df['timezone'] = zip(*df['Date'].apply(parse_date))
        df.drop(columns=['Date'], inplace=True)

    # add a new column 'email_type' to the dataframe, which represents the type of email (Foward, Reply, Normal)
    df['email_type'] = df['Subject'].apply(get_email_type)

        
    df['From'] = df['From'].map(split_email_addresses)
    df['To'] = df['To'].map(split_email_addresses)
    
    return df

In [11]:
process_dataframe(enron_df)
transform_dataframe(enron_df, split_date=False)

enron_df.head(10)

Number of rows with null values in 'From': 0
Number of rows with null values in 'To': 21847
Dropped 21847 rows. Remaining rows: 495554


,Date,From,To,X-To,X-From,X-cc,X-bcc,Subject,email_body,verdict,violated_rules,email_type
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),tim belden <tim belden/enron@enronxgate>,phillip k allen,NaN,NaN,,Here is our forecast\n\n,ALLOW,,Normal
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),john j lavorato <john j lavorato/enron@enronxg...,phillip k allen,NaN,NaN,Re:,Traveling to have a business meeting takes the...,ALLOW,,Re
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),leah van arsdall,phillip k allen,NaN,NaN,Re: test,test successful. way to go!!!,ALLOW,,Re
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),randall l gay,phillip k allen,NaN,NaN,,"Randy,\n\n Can you send me a schedule of the s...",ALLOW,,Normal
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),greg piper,phillip k allen,NaN,NaN,Re: Hello,Let's shoot for Tuesday at 11:45.,ALLOW,,Re
5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),greg piper,phillip k allen,NaN,NaN,Re: Hello,"Greg,\n\n How about either next Tuesday or Thu...",ALLOW,,Re
6,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",(phillip.allen@enron.com),"(david.l.johnson@enron.com, john.shafer@enron....","david.l.johnson@enron.com, john shafer",phillip k allen,NaN,NaN,,Please cc the following distribution list with...,ALLOW,,Normal
7,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",(phillip.allen@enron.com),(joyce.teixeira@enron.com),joyce teixeira,phillip k allen,NaN,NaN,Re: PRC review - phone calls,any morning between 10 and 11:30,ALLOW,,Re
8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",(phillip.allen@enron.com),(mark.scott@enron.com),mark scott,phillip k allen,NaN,NaN,Re: High Speed Internet Access,1. login: pallen pw: ke9davis\n\n I don't thi...,BLOCK,2.3,Re
9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",(phillip.allen@enron.com),(zimam@enron.com),zimam@enron.com,phillip k allen,NaN,NaN,FW: fixed forward or other Collar floor gas pr...,---------------------- Forwarded by Phillip K ...,BLOCK,2.1,FW
